<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:300%;text-align:center;border-radius: 20px 20px;font-family:cursive">HELLO</h1>

<font size="4"><h4><center>In this notebook, we will work on the Recommendation Systems. We will provide product recommendations to users and product we choose with Association Rule Learning, Collaborative Filtering.</center></h4></font>

<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 20px 20px"> Contents</h1>

* [Recommendation Systems](#1)
    * [Association Rule Learning](#2)
        * [Business Problem](#3)
        * [Imports](#4)
        * [Data Understanding](#5)
            * [İmporting Data](#6)
            * [Descriptive Statistics](#7)
        * [Data Preparation](#8)
        * [Invoice-Product Matrix](#9)
            * [Association Rules](#10)
            * [Apriori Algorithm](#11)
        * [Recommended Products](#12)
    * [Collaborative Filtering](#13)
        * [Item-Based Collaborative Filtering](#14)
            * [Business Problem](#15)
            * [Data Understanding](#16)
                * [İmporting Data](#17)
                * [Descriptive Statistics](#18)
            * [Data Preparation](#19)
                * [Creating the User Movie Df](#20)
            * [Recommended Movie](#21)
        * [User-Based Collaborative Filtering](#22)
            * [Data Understanding](#23)
            * [Data Preparation](#24)
            * [Weighted Average Recommendation Score's Calculate](#25)
            * [Recommended Movie](#26)

<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:300%;text-align:center;border-radius: 20px 20px">Recommendation Systems</h1><a id="1"></a>

[](http://)<font size="4">With the increasing need for e-commerce plotforms, recommendation systems have become increasingly important. Companies such as Netflix and Amazon use recommendation systems to offer content that will appeal to customers' needs and interests. For example, the recommended movies when we want to watch movies on Netflix. When we visit the e-commerce site to buy a product, suggestions such as those who bought this product also bought them. There are numerous recommendations similar to these ingredients. We'll take a look at how these events happen with recommendation systems</font>



<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 20px 20px">ASSOCIATION RULE LEARNING RECOMMENDER</h1><a id="2"></a>

[](http://)<font size="4">Association Rule Learning (Arl), performs analysis with the data held by a company. As a result of these analyzes, they perform future-oriented behaviors by deducing the association rules in the data. For example, let's say we've added a product to our cart. At the same time, we may come across different product recommendations. Here's how these recommendations came about. The product that we buy and the product that is most bought and that has a relationship is seen.</font>

<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Business Problem</h1><a id="3"></a>

[](http://)<font size="4"> Recommendation should be made to users who add products to their cart. </font>

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Imports</h2><a id="4"></a>

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Data Understanding</h2><a id="5"></a>


<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:150%;border-radius: 5px 5px">Importing Data</h2><a id="6"></a>

In [ ]:
df = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv")

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Descriptive Statistics</h2><a id="7"></a>

In [ ]:
df.shape  # Dimension of dataframe

In [ ]:
df.dtypes  # Data type of each variable 

In [ ]:
df.head()  # First 5 observations of dataframe

In [ ]:
df.tail()  # Last 5 observations of dataframe

In [ ]:
# We use it to suppress outliers. 
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Data Preparation</h2><a id="8"></a>

In [ ]:
df.dropna(inplace=True)  # Remove missing observations from the data set
df = df[~df["Invoice"].str.contains("C", na=False)]  # Delete operation if it starts with C in "Invoice".
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

# Functions required to delete outliers
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [ ]:
# Let's look at the last dataset
df.head() 

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Invoice-Product Matrix</h2><a id="9"></a>

In [ ]:
# I continued by choosing Germany from the data set.
df_ge = df[df["Country"] == "Germany"]
df_ge.head()

In [ ]:
# To create the Invoice-Product Matrix, I put the variables "Invoice" in the rows and "StockCode" in the columns.
# I place "Quntity" at the intersection of these two variables. 
# I functionalized the transactions by saying give 1 to the product with the invoice number and 0 to the product that is not.

def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
ge_inv_pro_df = create_invoice_product_df(df_ge, id=True)
ge_inv_pro_df.head()

In [ ]:
# Let's want to access the names of the products whose StockCode we know.

def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
    
l = ["21987", "23235", "22747"]
[check_id(df_ge, product_id) for product_id in l]


<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Association Rules</h2><a id="10"></a>

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:150%;border-radius: 5px 5px">Apriori Algorithm</h2><a id="11"></a>

In [ ]:
# Possibilities of all possible product combinations
frequent_itemsets = apriori(ge_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False).head(20)

In [ ]:
# Enforcement of Association Rules:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.sort_values("lift", ascending=False).head(30)

In [ ]:
# Product recommendation for users in the cart to be done
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []

    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))

    recommendation_list = list({item for item_list in recommendation_list for item in item_list})

    return recommendation_list[:rec_count]


In [ ]:
# The name of the product
check_id(df_ge, "21987")

In [ ]:
# 3 product recommended
arl_recommender(rules, "21987", 3)

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Recommended Products</h2><a id="12"></a>

In [ ]:
[[check_id(df_ge, i) for i in arl_recommender(rules, product, 3)] for product in l]

<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 20px 20px">Collaborative Filtering</h1><a id="13"></a>

[](http://)<font size="4">
This filtering method usually relies on collecting and analyzing information about the user's behavior, activities or preferences and predicting what they will like based on their similarity with other users. E.g; If a customer has watched movies X, Y and Z, and a different user has watched movies Y, Z and W, these two users can watch movies that they have not watched due to similar viewings. The collaborative filtering method is based on this example. We will consider the Item-Based and User-Based recommendation systems with the collaborative filtering method.</font>


<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 20px 20px">Item-Based Collaborative Filtering</h1><a id="14"></a>

[](http://)<font size="4">
Item-Based collaborative filtering refers to the habits of liking products. With any movie, the movie with the highest correlation is recommended by finding movies that show similar behavior to that movie.</font>



<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Business Problem</h1><a id="15"></a>

[](http://)<font size="4"> We would like to make a product recommendation for the users we have identified. Let's implement this product recommendation using the Item-Based Collaborative filtering method. </font>

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Data Understanding</h2><a id="16"></a>


<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Importing Data</h2><a id="17"></a>

In [ ]:
movie = pd.read_csv("../input/movielens-20m-dataset/movie.csv")
rating = pd.read_csv("../input/movielens-20m-dataset/rating.csv")
df = movie.merge(rating, how="left", on="movieId")

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Descriptive Statistics</h2><a id="18"></a>

In [ ]:
df.shape  # Dimension of dataframe

In [ ]:
df.dtypes  # Data type of each variable 

In [ ]:
df.head()  # First 5 observations of dataframe

In [ ]:
df.tail()  # Last 5 observations of dataframe

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Data Preparation</h2><a id="19"></a>

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Creating the User Movie Df</h2><a id="20"></a>

In [ ]:
df["title"].nunique()  # unique number of movies

In [ ]:
df["title"].value_counts().head(10)  # how many comments each movie has

In [ ]:
rating_counts = pd.DataFrame(df["title"].value_counts())

# I don't use movies with less than 2000 reviews
rare_movies = rating_counts[rating_counts["title"] <= 2000].index

# Movies with over 2000 reviews
comon_movies = df[~df["title"].isin(rare_movies)]

In [ ]:
comon_movies.head()

In [ ]:
comon_movies.shape

In [ ]:
comon_movies["title"].nunique()  # unique number of movies

In [ ]:
user_movie_df = comon_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

In [ ]:
user_movie_df.head()

In [ ]:
# Item-Based Film Recomended

user = 108170
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]
movie_name = pd.Series(df[df["movieId"] == movie_id][["title"]].values[0])

# Random choosing movies
# movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
# movie_name = user_movie_df[movie_name]

movies_from_item_based = user_movie_df.corrwith(movie_name).sort_values(ascending=False)

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Recommended Movie</h2><a id="21"></a>

In [ ]:
movies_from_item_based[1:6].index

<h1 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 20px 20px">User-Based Collaborative Filtering</h1><a id="22"></a>

[](http://)<font size="4">
User-Based Collaborative Filtering, a recommendation system is developed over a user who has similar liking behavior and watched the same movies as the user to whom advice is requested. For example, let's say users A and B. Let a user watch 12 movies. Let's assume that user B watched 10 of the movies A watched and gave similar scores. When B wants to watch a new movie, I can recommend movies that user A hasn't watched.</font>

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Data Understanding</h2><a id="23"></a>

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Data Preparation</h2><a id="24"></a>

In [ ]:
def create_user_movie_df(dataframe):
    comment_counts = pd.DataFrame(dataframe["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 2000].index
    comon_movies = dataframe[~dataframe["title"].isin(rare_movies)]
    user_movie_df = comon_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df(df)
user_movie_df.head()

In [ ]:
# Random choosing user
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

In [ ]:
# Movies watched by randomly selected user
random_user_df = user_movie_df[user_movie_df.index == random_user]
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

movies_watched[0:5]

In [ ]:
len(movies_watched)

In [ ]:
# Accessing Data and Ids of Other Users Watching the Same Movies

movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.shape

In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()  # total number of movies watched by each user
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
users_same_movies = user_movie_count[user_movie_count["movie_count"] > len(movies_watched) * 0.7]["userId"]
users_same_movies.count()

In [ ]:
# Identification of Users with the Most Similar Behaviors to the User to be Suggested
# For this we will perform 3 steps:
# 1. We will aggregate the data of the selected User and other users.
# 2. We will create the correlation df.
# 3. We will find the most similar users (Top Users).

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])
final_df.head()

In [ ]:
final_df.shape

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()

In [ ]:
# Users with a correlation of 0.7 percent or more with the user we selected

top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] > 0.7)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

rating = pd.read_csv("../input/movielens-20m-dataset/rating.csv")
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]

In [ ]:
top_users_ratings.head()

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Weighted Average Recommendation Score's Calculate</h2><a id="25"></a>

In [ ]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

In [ ]:
# Grouping the score given to each movie
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()

# Ranking of values whose weight average is greater than 3.6
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.6].sort_values("weighted_rating", ascending=False)
movies_to_be_recommend_final = movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"].head()

<h2 style="background-color:#42c5c1;font-family:newtimeroman;font-size:200%;border-radius: 5px 5px">Recommended Movie</h2><a id="26"></a>

In [ ]:
movies_to_be_recommend_final